In [1]:
from torch.utils.data import DataLoader, IterableDataset
from main import RESDataset

import os
import argparse

DATASET_PATH = '/Users/antoniaboca/Downloads/split-by-cath-topology/data'

parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, default='gvp', choices=['gvp'])
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--n_layers', type=int, default=5)
parser.add_argument('--gpus', type=int, default=0)
parser.add_argument('--data_file', type=str, default=DATASET_PATH)
parser.add_argument('--data_workers', type=int, default=0)
parser.add_argument('--max_len', type=int, default=None)
parser.add_argument('--sample_per_item', type=int, default=None)

args = parser.parse_args(['--model', 'gvp'])

train_dataloader = DataLoader(RESDataset(os.path.join(args.data_file, 'train'), shuffle=True, 
                        max_len=args.max_len, sample_per_item = args.sample_per_item), 
                        batch_size=None, num_workers=args.data_workers)
val_dataloader = DataLoader(RESDataset(os.path.join(args.data_file, 'val'), 
                    max_len=args.max_len, sample_per_item = args.sample_per_item), 
                    batch_size=None, num_workers=args.data_workers)
test_dataloader = DataLoader(RESDataset(os.path.join(args.data_file, 'test'), 
                    max_len=args.max_len, sample_per_item = args.sample_per_item), 
                    batch_size=None, num_workers=args.data_workers)

In [2]:
gen = iter(train_dataloader)

In [3]:
graph = next(gen)

In [4]:
graph

Data(x=[809, 3], edge_index=[2, 12920], node_type=[809], edge_s=[12920, 16], edge_v=[12920, 1, 3], label=10, ca_idx=40, ensemble='4ogg')

In [5]:
next(gen)

Data(x=[972, 3], edge_index=[2, 15964], node_type=[972], edge_s=[15964, 16], edge_v=[15964, 1, 3], label=9, ca_idx=45, ensemble='4ogg')

In [6]:
next(gen)

Data(x=[966, 3], edge_index=[2, 15478], node_type=[966], edge_s=[15478, 16], edge_v=[15478, 1, 3], label=4, ca_idx=41, ensemble='4ogg')

In [7]:
next(gen)

Data(x=[1014, 3], edge_index=[2, 15780], node_type=[1014], edge_s=[15780, 16], edge_v=[15780, 1, 3], label=12, ca_idx=61, ensemble='4ogg')